# SCREENER

Importing packages

In [6]:

# ========================
# Data Import and Handling
# ========================
import pandas as pd  # For data manipulation and analysis
import requests  # For making HTTP requests

# =======================
# Date and Time Handling
# =======================
from datetime import datetime, timedelta  # For working with dates and times

# ==============================
# Technical Analysis Indicators
# ==============================
import talib  # For technical analysis indicators

# ===================
# Plotting with Matplotlib
# ===================
import matplotlib.pyplot as plt  # For plotting using Matplotlib

# ======================
# Interactive Plotting
# ======================
import plotly.graph_objects as go  # For interactive plotting with Plotly

# ============================
# Statistical Data Visualization
# ============================
import seaborn as sns  # For statistical data visualization

# ==============================
# Displaying HTML in IPython
# ==============================
from IPython.display import HTML  

# ===============================
# Financial Charting with mplfinance
# ===============================
import mplfinance as mpf  # For financial charting with Matplotlib



In [7]:
API_URL = 'https://api.binance.com/api/v3/klines' # Binance API endpoint for candlestick data

## Functions

In [12]:
# ==============================
# RETRIEVING DATA FROM API
# ==============================

#The variable days_back represents the number of days in the past from which I intend to retrieve the data.
#The variable symbol is a coin pair. For example if you want bitcoin historical data, the symbol to use is BTCUSDT. 


def get_historical_data(symbol, days_back, interval):
    # Calculate the start time in milliseconds
    start_time = int((datetime.now() - timedelta(days=days_back)).timestamp() * 1000)

    # Calculate the current time in milliseconds
    end_time = int(datetime.now().timestamp() * 1000)

    # Make an API request to retrieve historical data
    response = requests.get(API_URL, params={
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000  # Maximum number of data points per request
    })

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # API request was successful
        hist_json = response.json()

        # Making a df from response
        # Extract only the relevant elements for each row
        df = pd.DataFrame(hist_json, columns=[
            'Time', 'Open Price', 'High Price', 'Low Price',
            'Close Price', 'Volume', 'Kline Close Time', 'Quote Asset Volume',
            'Number of Trades', 'Taker Buy Base Asset Volume',
            'Taker Buy Quote Asset Volume', 'Unused Field'
        ])

        # Select only the relevant columns (Open Time, Open Price, High Price, Low Price, Close Price, Volume)
        df = df[['Time', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume']]
        df['dateTime'] = pd.to_datetime(df['Time'], unit='ms')
        print("Retrived data for: " + symbol)

        return df
    else:
        # API request was not successful, print an error message
        print(f"Error: API request failed with status code {response.status_code}")
        return None

# ==============================
# TRANSFORM DATA
# ==============================

def transform_hist_data(df_name):
    df_name['Open Price'] = pd.to_numeric(df_name['Open Price'], errors='coerce').astype(float)
    df_name['High Price'] = pd.to_numeric(df_name['High Price'], errors='coerce').astype(float)
    df_name['Low Price'] = pd.to_numeric(df_name['Low Price'], errors='coerce').astype(float)
    df_name['Close Price'] = pd.to_numeric(df_name['Close Price'], errors='coerce').astype(float)
    df_name['Volume'] = pd.to_numeric(df_name['Volume'], errors='coerce').astype(float)
    return df_name

# ==============================
# CALCULATE EMA 12, 13 and 26
# ==============================
def calculate_emas(df_name):
    df_name['13EMA'] = df_name['Close Price'].ewm(span=13, adjust=False).mean()
    df_name['12EMA'] = df_name['Close Price'].ewm(span=12, adjust=False).mean()
    df_name['26EMA'] = df_name['Close Price'].ewm(span=26, adjust=False).mean()
    return df_name
    
# ==============================
# CALCULATE MACD 12, 26, 9
# ==============================

def calculate_macd(df_name):
    # Calculate MACD Line (12EMA - 26EMA)
    df_name['MACD'] = df_name['12EMA'] - df_name['26EMA']

    # Calculate Signal Line (9-period EMA of MACD)
    df_name['Signal'] = df_name['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate MACD Histogram
    df_name['Histogram'] = df_name['MACD'] - df_name['Signal']
    return df_name
    
# ==============================
# CALCULATE ELDER IMPULSE
# ==============================

def calculate_elder(df_name):

    for i in range(1, len(df_name)):
        ema_comparison = 0
        macd_comparison = 0
        first_row_ema = df_name.iloc[i-1]['13EMA']
        second_row_ema = df_name.iloc[i]['13EMA']
        if second_row_ema > first_row_ema:
            ema_comparison = 1
        elif second_row_ema < first_row_ema:
            ema_comparison = -1
        first_row_macd = df_name.iloc[i-1]['Histogram']
        second_row_macd = df_name.iloc[i]['Histogram']
        if second_row_macd > first_row_macd:
            macd_comparison = 1
        elif second_row_macd < first_row_macd:
            macd_comparison = -1
        if ema_comparison == 1 and macd_comparison == 1:
              df_name.at[i, 'elder_impulse'] = 1
        elif ema_comparison == -1 and macd_comparison == -1:
            df_name.at[i, 'elder_impulse'] = -1
        else:
            df_name.at[i, 'elder_impulse'] = 0
    return df_name
# ==============================
# CALCULATE ATR CHANNELS
# ==============================
def calculate_ATR(df_name):
    df_name['ATR21'] = talib.ATR(df_name['High Price'], df_name['Low Price'], df_name['Close Price'], timeperiod=21)
    df_name['21EMA'] = df_name['Close Price'].ewm(span=21, adjust=False).mean()
    df_name['ATR+1'] = df_name['21EMA'] + df_name['ATR21'] 
    df_name['ATR+2'] = df_name['21EMA'] + 2 * df_name['ATR21'] 
    df_name['ATR+3'] = df_name['21EMA'] + 3 * df_name['ATR21'] 
    df_name['ATR-1'] = df_name['21EMA'] - df_name['ATR21'] 
    df_name['ATR-2'] = df_name['21EMA'] - 2 * df_name['ATR21'] 
    df_name['ATR-3'] = df_name['21EMA'] - 3 * df_name['ATR21'] 
    return df_name
# ==============================
# TOP CRYPTO TO USDT LIST
# ==============================

def get_top_cryptos_with_usdt(number = 30):
    url = "https://api.binance.com/api/v3/ticker/24hr"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        symbol_count_dict = {entry.get('symbol', None): entry.get('count', None) for entry in data if 'USDT' in entry.get('symbol', '')}
        sorted_dict_values = dict(sorted(symbol_count_dict.items(), key=lambda item: item[1], reverse = True))
        top_crypto_usdt = list(sorted_dict_values.keys())[:number]
        return top_crypto_usdt
    else:
        print(f"Error: {response.status_code}")
        return None

# Run the function
top_cryptos_usdt = get_top_cryptos_with_usdt(100)




## Creating a table with elder impulse

Elder Impulse
https://school.stockcharts.com/doku.php?id=chart_analysis:elder_impulse_system
MACD-histogram
https://www.investopedia.com/ask/answers/122414/what-moving-average-convergence-divergence-macd-formula-and-how-it-calculated.asp

In [13]:
df_long = get_historical_data('BTCUSDT', 360, '1w')
df_long = transform_hist_data(df_long)
df_long = calculate_emas(df_long)
df_long = calculate_macd(df_long)
df_long = calculate_elder(df_long)

Retrived data for: BTCUSDT


In [14]:
df_long.tail(10)

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA,12EMA,26EMA,MACD,Signal,Histogram,elder_impulse
42,1704067200000,42283.58,45879.63,40750.00,43929.02,310487.48622,2024-01-01,38590.195710,38995.953657,35008.801205,3987.152452,2903.195983,1083.956469,0.0
43,1704672000000,43929.01,48969.48,41500.00,41732.35,470798.80434,2024-01-08,39039.074894,39416.937710,35506.841857,3910.095853,3104.575957,805.519896,0.0
44,1705276800000,41732.35,43578.01,40280.00,41580.33,238486.27274,2024-01-15,39402.111338,39749.767293,35956.729867,3793.037426,3242.268250,550.769175,0.0
45,1705881600000,41580.32,42842.68,38555.00,42031.06,274603.16207,2024-01-22,39777.675432,40100.735402,36406.680247,3694.055154,3332.625631,361.429523,0.0
46,1706486400000,42031.05,43882.36,41804.88,42582.88,203036.61925,2024-01-29,40178.418942,40482.603801,36864.176525,3618.427276,3389.785960,228.641316,0.0
47,1707091200000,42582.88,48592.66,42258.10,48299.99,262240.48357,2024-02-05,41338.643379,41685.278601,37711.273820,3974.004781,3506.629724,467.375057,1.0
48,1707696000000,48300.00,52816.62,47710.01,52137.67,310862.30170,2024-02-12,42881.361468,43293.338816,38779.895759,4513.443057,3707.992391,805.450666,1.0
49,1708300800000,52137.68,52985.00,50521.00,51728.85,223366.16186,2024-02-19,44145.288401,44591.109768,39739.077555,4852.032213,3936.800355,915.231858,1.0
50,1708905600000,51728.85,64000.00,50901.44,63113.97,417907.83383,2024-02-26,46855.100058,47440.780573,41470.551069,5970.229504,4343.486185,1626.743319,1.0
51,1709510400000,63113.97,67524.99,62300.00,67335.70,69899.23920,2024-03-04,49780.900050,50501.537408,43386.488027,7115.049381,4897.798824,2217.250557,1.0


if elder_impulse is 1, it means you can go long; 
if it is -1 , you can go short;
if it is 0 you can do both. 
It is best to go long if impulse is 1 or 0 after -1. (This should be considered good metricks for stock screening)

# SCREENER

- Leian top krüptod
- Laen iga krüpto kohta alla pika vaate
- Teen EMA ja MACD arvutused
- Teen Elder Impulse arvutused
- Värviliselt näitan ainult neid, kus peale punast tuleb roheline või sinine või siis peale rohelist tuleb sinine või punane


In [15]:


# Define colors
green_color = "#00cc00"
red_color = "#ff6666"

bull_set = set() #Creating a set for all bulls
bear_set = set() #Creating a set for all bears

for crypto in top_cryptos_usdt:
    #Teen arvutused ja panen tabelisse
    df = get_historical_data(crypto, 360, '1w')
    df = transform_hist_data(df)
    df = calculate_emas(df)
    df = calculate_macd(df)
    df = calculate_elder(df)
    #Kui elder on roheline või sinine peale punast, siis on pull
    if df.iloc[-2]['elder_impulse'] == -1 and (df.iloc[-1]['elder_impulse'] == 0 or df.iloc[-1]['elder_impulse'] == 1):
        bull_set.add(crypto)
        display(HTML(f"<span style='color:{green_color}'>Added {crypto} to bull_set</span>"))
    #Kui elder on punane või sinine peale rohelist, siis on karu
    elif df.iloc[-2]['elder_impulse'] == 1 and (df.iloc[-1]['elder_impulse'] == 0 or df.iloc[-1]['elder_impulse'] == -1):
        bear_set.add(crypto)
        display(HTML(f"<span style='color:{red_color}'>Added {crypto} to bear_set</span>"))
    else:
        print(crypto + " is not acceptable")
    time.sleep(0.1)
    
    
    

Retrived data for: SHIBUSDT
SHIBUSDT is not acceptable
Retrived data for: BTCUSDT
BTCUSDT is not acceptable
Retrived data for: PEPEUSDT
PEPEUSDT is not acceptable
Retrived data for: BONKUSDT
BONKUSDT is not acceptable
Retrived data for: FDUSDUSDT
FDUSDUSDT is not acceptable
Retrived data for: DOGEUSDT
DOGEUSDT is not acceptable
Retrived data for: FLOKIUSDT
FLOKIUSDT is not acceptable
Retrived data for: ETHUSDT
ETHUSDT is not acceptable
Retrived data for: MEMEUSDT
MEMEUSDT is not acceptable
Retrived data for: SOLUSDT
SOLUSDT is not acceptable
Retrived data for: 1000SATSUSDT
1000SATSUSDT is not acceptable
Retrived data for: JASMYUSDT
JASMYUSDT is not acceptable
Retrived data for: ORDIUSDT
ORDIUSDT is not acceptable
Retrived data for: FTMUSDT
FTMUSDT is not acceptable
Retrived data for: LUNCUSDT
LUNCUSDT is not acceptable
Retrived data for: PEOPLEUSDT
PEOPLEUSDT is not acceptable
Retrived data for: WLDUSDT
WLDUSDT is not acceptable
Retrived data for: QTUMUSDT
QTUMUSDT is not acceptable
Re

Retrived data for: LEVERUSDT
LEVERUSDT is not acceptable
Retrived data for: APTUSDT
APTUSDT is not acceptable
Retrived data for: USDCUSDT
USDCUSDT is not acceptable
Retrived data for: GALAUSDT
GALAUSDT is not acceptable
Retrived data for: NEARUSDT
NEARUSDT is not acceptable
Retrived data for: ACEUSDT
ACEUSDT is not acceptable
Retrived data for: HOTUSDT
HOTUSDT is not acceptable
Retrived data for: XAIUSDT


Retrived data for: ARKMUSDT
ARKMUSDT is not acceptable
Retrived data for: SLPUSDT
SLPUSDT is not acceptable
Retrived data for: ALTUSDT
ALTUSDT is not acceptable
Retrived data for: EPXUSDT
EPXUSDT is not acceptable
Retrived data for: CRVUSDT
CRVUSDT is not acceptable
Retrived data for: ENJUSDT
ENJUSDT is not acceptable
Retrived data for: AVAXUSDT


Retrived data for: NFPUSDT
NFPUSDT is not acceptable
Retrived data for: SXPUSDT
SXPUSDT is not acceptable
Retrived data for: LTCUSDT
LTCUSDT is not acceptable
Retrived data for: NEOUSDT
NEOUSDT is not acceptable
Retrived data for: WAVESUSDT
WAVESUSDT is not acceptable
Retrived data for: JUPUSDT
JUPUSDT is not acceptable
Retrived data for: RNDRUSDT
RNDRUSDT is not acceptable
Retrived data for: CFXUSDT
CFXUSDT is not acceptable
Retrived data for: INJUSDT


Retrived data for: ATOMUSDT
ATOMUSDT is not acceptable
Retrived data for: STXUSDT
STXUSDT is not acceptable
Retrived data for: ICPUSDT
ICPUSDT is not acceptable
Retrived data for: ARUSDT
ARUSDT is not acceptable
Retrived data for: APEUSDT
APEUSDT is not acceptable
Retrived data for: PIXELUSDT
PIXELUSDT is not acceptable
Retrived data for: SPELLUSDT
SPELLUSDT is not acceptable
Retrived data for: BADGERUSDT
BADGERUSDT is not acceptable
Retrived data for: MANTAUSDT
MANTAUSDT is not acceptable
Retrived data for: PERPUSDT
PERPUSDT is not acceptable
Retrived data for: OCEANUSDT
OCEANUSDT is not acceptable
Retrived data for: MOVRUSDT
MOVRUSDT is not acceptable
Retrived data for: COTIUSDT
COTIUSDT is not acceptable
Retrived data for: SANDUSDT
SANDUSDT is not acceptable
Retrived data for: BTTCUSDT
BTTCUSDT is not acceptable
Retrived data for: BEAMXUSDT
BEAMXUSDT is not acceptable
Retrived data for: UNIUSDT
UNIUSDT is not acceptable
Retrived data for: KDAUSDT
KDAUSDT is not acceptable
Retrived d

Retrived data for: PORTALUSDT
PORTALUSDT is not acceptable
Retrived data for: MINAUSDT
MINAUSDT is not acceptable
Retrived data for: MDTUSDT
MDTUSDT is not acceptable
Retrived data for: EDUUSDT
EDUUSDT is not acceptable
Retrived data for: ROSEUSDT
ROSEUSDT is not acceptable
Retrived data for: GASUSDT
GASUSDT is not acceptable
Retrived data for: VETUSDT
VETUSDT is not acceptable
Retrived data for: VIDTUSDT


Retrived data for: TIAUSDT
TIAUSDT is not acceptable
Retrived data for: ALGOUSDT
ALGOUSDT is not acceptable
Retrived data for: IMXUSDT


In [16]:
bear_set

{'AVAXUSDT',
 'IMXUSDT',
 'INJUSDT',
 'LINKUSDT',
 'RAYUSDT',
 'VIDTUSDT',
 'XAIUSDT'}

In [17]:
bull_set

set()

## Creating Medium view and ATR channels

In [25]:
df_medium = get_historical_data('BTCUSDT', 60, '1d')
df_medium = calculate_ATR(df_medium)
df_medium = transform_hist_data(df_medium)

Retrived data for: BTCUSDT


In [27]:
df_medium.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Time         60 non-null     int64         
 1   Open Price   60 non-null     float64       
 2   High Price   60 non-null     float64       
 3   Low Price    60 non-null     float64       
 4   Close Price  60 non-null     float64       
 5   Volume       60 non-null     float64       
 6   dateTime     60 non-null     datetime64[ns]
 7   ATR21        39 non-null     float64       
 8   21EMA        60 non-null     float64       
 9   ATR+1        39 non-null     float64       
 10  ATR+2        39 non-null     float64       
 11  ATR+3        39 non-null     float64       
 12  ATR-1        39 non-null     float64       
 13  ATR-2        39 non-null     float64       
 14  ATR-3        39 non-null     float64       
dtypes: datetime64[ns](1), float64(13), int64(1)
memory usage: 7

In [21]:
df_medium.head()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,ATR21,21EMA,ATR+1,ATR+2,ATR+3,ATR-1,ATR-2,ATR-3
0,1704412800000,44151.10,44357.46,42450.00,44145.11,48075.25327,2024-01-05,NaN,44145.110000,NaN,NaN,NaN,NaN,NaN,NaN
1,1704499200000,44145.12,44214.42,43397.05,43968.32,17835.06144,2024-01-06,NaN,44129.038182,NaN,NaN,NaN,NaN,NaN,NaN
2,1704585600000,43968.32,44480.59,43572.09,43929.02,23023.85080,2024-01-07,NaN,44110.854711,NaN,NaN,NaN,NaN,NaN,NaN
3,1704672000000,43929.01,47248.99,43175.00,46951.04,72814.57589,2024-01-08,NaN,44369.053373,NaN,NaN,NaN,NaN,NaN,NaN
4,1704758400000,46951.04,47972.00,44748.67,46110.00,69927.66617,2024-01-09,NaN,44527.321249,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# FUNCTION for drawing medium plotly graph

def draw_ATR_GRAPH(df_name):


    # Create a candlestick trace
    df_name['formatted_date'] = df_name['dateTime'].dt.strftime('%m-%d')
    candlestick = go.Candlestick(x=df_name['formatted_date'],
                                open=df_name['Open Price'],
                                high=df_name['High Price'],
                                low=df_name['Low Price'],
                                close=df_name['Close Price'],
                                name='Candlestick')

    # 21 EMA trace
    line_trace_21_EMA = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['21EMA'],
                        mode='lines',
                        name='21EMA',
                        line=dict(color= 'black'))
    # ATR +1 trace
    line_trace_ATR_1 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+1'],
                        mode='lines',
                        name='ATR+1',
                        line=dict(color= 'green'))

    # ATR +2 trace
    line_trace_ATR_2 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+2'],
                        mode='lines',
                        name='ATR+2',
                        line=dict(color= 'orange'))

    # ATR +1 trace
    line_trace_ATR_1 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+1'],
                        mode='lines',
                        name='ATR+1',
                        line=dict(color= 'green'))

    # ATR +3 trace
    line_trace_ATR_3 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR+3'],
                        mode='lines',
                        name='ATR+3',
                        line=dict(color= 'red'))

    # ATR -1 trace
    line_trace_ATR_neg1 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR-1'],
                        mode='lines',
                        name='ATR-1',
                        line=dict(color= 'green'))
    # ATR -2 trace
    line_trace_ATR_neg2 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR-2'],
                        mode='lines',
                        name='ATR-2',
                        line=dict(color= 'orange'))
    # ATR -3 trace
    line_trace_ATR_neg3 = go.Scatter(x=df_name['formatted_date'],
                        y=df_name['ATR-3'],
                        mode='lines',
                        name='ATR-3',
                        line=dict(color= 'red'))

    # Create layout
    layout = go.Layout(title='Candlestick Chart with Extra Line',
                    xaxis=dict(type='category', categoryorder='category ascending'),
                    yaxis=dict(title='Price'))

    # Create figure

    fig = go.Figure(data=[candlestick, line_trace_21_EMA, line_trace_ATR_1, line_trace_ATR_2, line_trace_ATR_3, line_trace_ATR_neg1, line_trace_ATR_neg2, line_trace_ATR_neg3], layout=layout)
    return fig


In [48]:
draw_ATR_GRAPH(df_medium)

Siit lihtsalt testimine. Salvestab ilusti htmlina ära. 
Siit edasi:
1. tee tsükkel, kus teeb html-id nendest, mis on pullid ja bearid
2. saada need meilile

In [46]:
import os
import plotly.io as pio
# Output directory where HTML files will be saved
output_directory = '/Users/ahtojarve/Trading_bot/reports/figures/'
os.makedirs(output_directory, exist_ok=True)
chart_filename = f"{output_directory}_testing.html"
pio.write_html(fig, file = chart_filename)

# Päevase vaate graafikkute tegemine
- vastavalt bull ja bear setile joonistan graafikud endale välja, kus on peal siis vajalikud jooned
- teen ikkagilt ise otsuse
- esitan orderi
- (PÄRAST ON VAJA KUIDAGI SEE BACTESTIMINE SELGEKS SAADA)